In [43]:
import pickle
import tensorflow
from keras import Model
from keras.layers import (
    Input,
    TimeDistributed,
    LSTM,
    Dense,
    Input,
    Dense,
    Conv1D,
    Flatten, 
    Embedding,
    Dropout, 
    Bidirectional)
from imblearn.over_sampling import SMOTE
import numpy
from numba import cuda 
import tensorflow as tf
import tensorflow_models as tfm

device = cuda.get_current_device()
device.reset()
%run constants.py


In [44]:

#Output window override (so you can predict on more ouput windows)
TIME_STEP: int = 90
output: int = 60

# Some file name editing for timestep
f = os.path.basename(X_TRAIN_SINGLE_TIMESTEP_RAW)
X_TRAIN_SINGLE_TIMESTEP_RAW = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(TIME_STEP) + os.path.splitext(os.path.basename(f))[1]) 
f = os.path.basename(X_TEST_SINGLE_TIMESTEP_RAW)
X_TEST_SINGLE_TIMESTEP_RAW = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(TIME_STEP) + os.path.splitext(os.path.basename(f))[1]) 
f = os.path.basename(X_VAL_SINGLE_TIMESTEP_RAW)
X_VAL_SINGLE_TIMESTEP_RAW = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(TIME_STEP) + os.path.splitext(os.path.basename(f))[1]) 


f = os.path.basename(X_TRAIN_INPUT_SAVE_FILE_VEC_SINGLE_TIMESTEP)
X_TRAIN_INPUT_SAVE_FILE_VEC_SINGLE_TIMESTEP = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(TIME_STEP) + os.path.splitext(os.path.basename(f))[1]) 
f = os.path.basename(X_TEST_INPUT_SAVE_FILE_VEC_SINGLE_TIMESTEP)
X_TEST_INPUT_SAVE_FILE_VEC_SINGLE_TIMESTEP = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(TIME_STEP) + os.path.splitext(os.path.basename(f))[1]) 
f = os.path.basename(X_VAL_INPUT_SAVE_FILE_VEC_SINGLE_TIMESTEP)
X_VAL_INPUT_SAVE_FILE_VEC_SINGLE_TIMESTEP = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(TIME_STEP) + os.path.splitext(os.path.basename(f))[1]) 

f = os.path.basename(X_TRAIN_INPUT_SAVE_FILE_VEC_MULTI_TIMESTEP)
X_TRAIN_INPUT_SAVE_FILE_VEC_MULTI_TIMESTEP = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(TIME_STEP) + os.path.splitext(os.path.basename(f))[1]) 
f = os.path.basename(X_TEST_INPUT_SAVE_FILE_VEC_MULTI_TIMESTEP)
X_TEST_INPUT_SAVE_FILE_VEC_MULTI_TIMESTEP = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(TIME_STEP) + os.path.splitext(os.path.basename(f))[1]) 
f = os.path.basename(X_VAL_INPUT_SAVE_FILE_VEC_MULTI_TIMESTEP)
X_VAL_INPUT_SAVE_FILE_VEC_MULTI_TIMESTEP = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(TIME_STEP) + os.path.splitext(os.path.basename(f))[1]) 

f = os.path.basename(Y_TRAIN_INPUT_SAVE_FILE)
Y_TRAIN_INPUT_SAVE_FILE = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(output) + os.path.splitext(os.path.basename(f))[1]) 
f = os.path.basename(Y_TEST_INPUT_SAVE_FILE)
Y_TEST_INPUT_SAVE_FILE = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(output) + os.path.splitext(os.path.basename(f))[1]) 
f = os.path.basename(Y_VAL_INPUT_SAVE_FILE)
Y_VAL_INPUT_SAVE_FILE = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(output) + os.path.splitext(os.path.basename(f))[1]) 

f = os.path.basename(EMBEDDING_MATRIX_SAVE_FILE)
EMBEDDING_MATRIX_SAVE_FILE= os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(TIME_STEP) + os.path.splitext(os.path.basename(f))[1]) 
f = os.path.basename(VOCAB_SAVE_FILE)
VOCAB_SAVE_FILE = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(TIME_STEP) + os.path.splitext(os.path.basename(f))[1]) 
f = os.path.basename(TRAIN_CORPORA)
TRAIN_CORPORA = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(TIME_STEP) + os.path.splitext(os.path.basename(f))[1]) 

file_suffix: str = f'_i{TIME_STEP}_o{output}'



In [45]:
with open(X_TRAIN_INPUT_SAVE_FILE_VEC_MULTI_TIMESTEP, "rb") as f:
        x_train = pickle.load(f)
with open(Y_TRAIN_INPUT_SAVE_FILE, "rb") as f:
        y_train = pickle.load(f)
with open(X_TEST_INPUT_SAVE_FILE_VEC_MULTI_TIMESTEP, "rb") as f:
        x_test = pickle.load(f)
with open(Y_TEST_INPUT_SAVE_FILE, "rb") as f:
        y_test = pickle.load(f)
with open(X_VAL_INPUT_SAVE_FILE_VEC_MULTI_TIMESTEP, "rb") as f:
        x_val = pickle.load(f)
with open(Y_VAL_INPUT_SAVE_FILE, "rb") as f:
        y_val = pickle.load(f)
with open(EMBEDDING_MATRIX_SAVE_FILE, "rb") as f:
        embedding_matrix = pickle.load(f)

print("------Saving varaibles for reuse ------")
print(f"X_train shape: {x_train.shape}")
print(f"Y_train shape: {y_train.shape}")
print(f"X_test shape: {x_test.shape}")
print(f"Y_test shape: {y_test.shape}")
print(f"X_val shape: {x_val.shape}")
print(f"Y_val shape: {y_val.shape}")
print(f"Embedding shape: {embedding_matrix.shape}")
print(f"Total 0 values: {(y_train == 0).sum()}")
print(f"Total 1 values: {(y_train == 1).sum()}")

------Saving varaibles for reuse ------
X_train shape: (2465, 90, 200)
Y_train shape: (2465,)
X_test shape: (685, 90, 200)
Y_test shape: (685,)
X_val shape: (274, 90, 200)
Y_val shape: (274,)
Embedding shape: (10000, 100)
Total 0 values: 1151
Total 1 values: 1314


In [46]:
if BALANCE_DATA:
    arr = x_train.reshape(len(x_train), -1)
    print(arr.shape)
    sm = SMOTE(random_state=SEED)
    x_train_bal, y_train = sm.fit_resample(arr, y_train.ravel())
    print(f"After OverSampling, the shape of train_X: {x_train_bal.shape}")
    print(f"After OverSampling, the shape of train_y: {y_train.shape}")
    print(f"After OverSampling, counts of label '1': {sum(y_train == 1)}")
    print(f"After OverSampling, counts of label '0': {sum(y_train == 0)}")
    x_train = numpy.reshape(x_train_bal, (-1, x_train.shape[1], x_train.shape[2]))
x_train.shape


(2465, 18000)
After OverSampling, the shape of train_X: (2628, 18000)
After OverSampling, the shape of train_y: (2628,)
After OverSampling, counts of label '1': 1314
After OverSampling, counts of label '0': 1314


(2628, 90, 200)

In [47]:
# embedding_layer = Embedding(
#         MAX_VOCAB_SIZE,
#         EMBEDDING_DIM,
#         weights=[embedding_matrix],
#         input_length=MAX_SEQUENCE_LENGTH,
#         trainable=False,)

# x_train= tensorflow.convert_to_tensor(x_train)
# y_train= tensorflow.convert_to_tensor(y_train)

# optimizer = tensorflow.keras.optimizers.Adam(learning_rate=LR)
# loss = tf.keras.losses.BinaryCrossentropy()
# metrics = tf.metrics.BinaryAccuracy()

In [48]:
# import matplotlib.pyplot as plt
# def plot_graph(history):
#     history_dict = history.history
#     print(history_dict.keys())

#     acc = history_dict['binary_accuracy']
#     val_acc = history_dict['val_binary_accuracy']
#     loss = history_dict['loss']
#     val_loss = history_dict['val_loss']

#     epochs = range(1, len(acc) + 1)
#     fig = plt.figure(figsize=(10, 6))
#     fig.tight_layout()

#     plt.subplot(2, 1, 1)
#     # r is for "solid red line"
#     plt.plot(epochs, loss, 'r', label='Training loss')
#     # b is for "solid blue line"
#     plt.plot(epochs, val_loss, 'b', label='Validation loss')
#     plt.title('Training and validation loss')
#     # plt.xlabel('Epochs')
#     plt.ylabel('Loss')
#     plt.legend()

#     plt.subplot(2, 1, 2)
#     plt.plot(epochs, acc, 'r', label='Training acc')
#     plt.plot(epochs, val_acc, 'b', label='Validation acc')
#     plt.title('Training and validation accuracy')
#     plt.xlabel('Epochs')
#     plt.ylabel('Accuracy')
#     plt.legend(loc='lower right')
#     return plt

# STACKED LSTM CLASSIFIER

In [49]:
# def run_stacked_lstm():
#     document_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int64")
#     embedding_sequences =embedding_layer(document_input)
#     x = LSTM(TIME_STEP, return_sequences=True)(embedding_sequences)
#     x = LSTM(TIME_STEP)(x)
#     x = Dropout(0.3)(x)
#     doc_model = Model(document_input, x)
#     input_docs = Input(
#                 shape=(TIME_STEP, MAX_SEQUENCE_LENGTH), name="input_docs", dtype="int32"
#             )

#     x = TimeDistributed(doc_model, name="token_embedding_model")(input_docs)
#     x = LSTM(TIME_STEP)(x)
#     x = Dropout(0.3)(x)
#     outputs = Dense(1, activation="sigmoid")(x)
#     model = Model(input_docs, outputs)

#     model.summary()
#     return model

In [50]:
# model = run_stacked_lstm()
# model.compile(optimizer=optimizer,
#                          loss=loss,
#                          metrics=metrics)
# history = model.fit(x=x_train, y=y_train,
#                                 validation_data = (x_val, y_val),
#                                epochs=NUM_EPOCHS)

In [51]:
# loss, accuracy = model.evaluate(x= x_test, y= y_test)
# print(f'Loss: {loss}')
# print(f'Accuracy: {accuracy}')
# graph = plot_graph(history)
# save_file_name: str = f'saved_graphs/multi_timestep_lstm_stacked{file_suffix}.png' 
# graph.savefig(save_file_name)
# model_name: str = f'saved_model/multi_timestep_lstm_stacked{file_suffix}' 
# model.save(model_name)

# LSTM Classifier

In [52]:
# def run_lstm():
#     document_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
#     embedding_sequences =embedding_layer(document_input)
#     x = LSTM(TIME_STEP)(embedding_sequences)
#     x = Dropout(0.3)(x)
#     doc_model = Model(document_input, x)
#     input_docs = Input(
#                 shape=(TIME_STEP, MAX_SEQUENCE_LENGTH), name="input_docs", dtype="int32"
#             )

#     x = TimeDistributed(doc_model, name="token_embedding_model")(input_docs)
#     x = LSTM(TIME_STEP)(x)
#     x = Dropout(0.3)(x)
#     outputs = Dense(1, activation="sigmoid")(x)
#     model = Model(input_docs, outputs)
#     model.summary()
#     return model

In [53]:
# model = run_lstm()

# optimizer = tensorflow.keras.optimizers.Adam(learning_rate=LR)
# loss = tf.keras.losses.BinaryCrossentropy()
# metrics = tf.metrics.BinaryAccuracy()
# model.compile(optimizer=optimizer,
#                          loss=loss,
#                          metrics=metrics)
# history = model.fit(x=x_train, y=y_train,
#                                 validation_data = (x_val, y_val),
#                                epochs=NUM_EPOCHS)

In [54]:
# loss, accuracy = model.evaluate(x= x_test, y= y_test)
# print(f'Loss: {loss}')
# print(f'Accuracy: {accuracy}')
# graph = plot_graph(history)
# save_file_name: str = f'saved_graphs/multi_timestep_lstm{file_suffix}.png' 
# graph.savefig(save_file_name)
# model_name: str = f'saved_model/multi_timestep_lstm{file_suffix}' 
# model.save(model_name)

# CNN Classifier

In [55]:
# def run_cnn():
#     document_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
#     embedding_sequences = embedding_layer(document_input)

#     x = Conv1D(filters=400, kernel_size=5, padding="valid")(embedding_sequences)
#     x = Dropout(0.3)(x)
#     doc_model = Model(document_input, x)

#     input_docs = Input(
#             shape=(TIME_STEP, MAX_SEQUENCE_LENGTH), name="input_docs", dtype="int32"
#         )

#     x = TimeDistributed(doc_model, name="token_embedding_model")(input_docs)
#     x = Conv1D(filters=400, kernel_size=5, padding="valid")(x)
#     x = Dropout(0.3)(x)
#     x = Flatten()(x)
#     outputs = Dense(1, activation="sigmoid")(x)

#     model = Model(input_docs, outputs)
#     return model

In [56]:
# model = run_cnn()

# optimizer = tensorflow.keras.optimizers.Adam(learning_rate=LR)
# loss = tf.keras.losses.BinaryCrossentropy()
# metrics = tf.metrics.BinaryAccuracy()
# model.compile(optimizer=optimizer,
#                          loss=loss,
#                          metrics=metrics)
# history = model.fit(x=x_train, y=y_train,
#                                 validation_data = (x_val, y_val),
#                                epochs=NUM_EPOCHS)

In [57]:
# loss, accuracy = model.evaluate(x= x_test, y= y_test)
# print(f'Loss: {loss}')
# print(f'Accuracy: {accuracy}')
# graph = plot_graph(history)
# save_file_name: str = f'saved_graphs/multi_timestep_cnn{file_suffix}.png' 
# graph.savefig(save_file_name)
# model_name: str = f'saved_model/multi_timestep_cnn{file_suffix}' 
# model.save(model_name)

# BiLSTM Classifier

In [58]:
# def run_bilstm():
#     document_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
#     embedding_sequences =embedding_layer(document_input)
#     x = Bidirectional(LSTM(TIME_STEP))(embedding_sequences)
#     x = Dropout(0.3)(x)
#     doc_model = Model(document_input, x)
#     input_docs = Input(
#                 shape=(TIME_STEP, MAX_SEQUENCE_LENGTH), name="input_docs", dtype="int32"
#             )

#     x = TimeDistributed(doc_model, name="token_embedding_model")(input_docs)
#     x = Bidirectional(LSTM(TIME_STEP))(x)
#     x = Dropout(0.3)(x)
#     outputs = Dense(1, activation="sigmoid")(x)
#     model = Model(input_docs, outputs)
#     model.summary()
#     return model

In [59]:
# model = run_bilstm()

# optimizer = tensorflow.keras.optimizers.Adam(learning_rate=LR)
# loss = tf.keras.losses.BinaryCrossentropy()
# metrics = tf.metrics.BinaryAccuracy()
# model.compile(optimizer=optimizer,
#                          loss=loss,
#                          metrics=metrics)
# history = model.fit(x=x_train, y=y_train,
#                                 validation_data = (x_val, y_val),
#                                epochs=NUM_EPOCHS)

In [60]:
# loss, accuracy = model.evaluate(x= x_test, y= y_test)
# print(f'Loss: {loss}')
# print(f'Accuracy: {accuracy}')
# graph = plot_graph(history)
# save_file_name: str = f'saved_graphs/multi_timestep_bilstm{file_suffix}.png' 
# graph.savefig(save_file_name)
# model_name: str = f'saved_model/multi_timestep_bilstm{file_suffix}' 
# model.save(model_name)

# BiLSTM Stacked Classifier

In [61]:
# def run_bilstm_stacked():
#     document_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
#     embedding_sequences =embedding_layer(document_input)
#     x = Bidirectional(LSTM(60, return_sequences=True))(embedding_sequences)
#     x = Bidirectional(LSTM(60))(x)
#     doc_model = Model(document_input, x)
#     input_docs = Input(
#                 shape=(TIME_STEP, MAX_SEQUENCE_LENGTH), name="input_docs", dtype="int32"
#             )

#     x = TimeDistributed(doc_model, name="token_embedding_model")(input_docs)
#     x = Bidirectional(LSTM(60))(x)
#     x = Dropout(0.3)(x)
#     outputs = Dense(1, activation="sigmoid")(x)
#     model = Model(input_docs, outputs)
#     model.summary()
#     return model

In [62]:
# model = run_bilstm_stacked()

# optimizer = tensorflow.keras.optimizers.Adam(learning_rate=LR)
# loss = tf.keras.losses.BinaryCrossentropy()
# metrics = tf.metrics.BinaryAccuracy()
# model.compile(optimizer=optimizer,
#                          loss=loss,
#                          metrics=metrics)
# history = model.fit(x=x_train, y=y_train,
#                                 validation_data = (x_val, y_val),
#                                epochs=NUM_EPOCHS)

In [63]:
# loss, accuracy = model.evaluate(x= x_test, y= y_test)
# print(f'Loss: {loss}')
# print(f'Accuracy: {accuracy}')
# graph = plot_graph(history)
# save_file_name: str = f'saved_graphs/stacked_bilstm_i{TIME_STEP}_o{output}.png' 
# graph.savefig(save_file_name)
# model_name: str = f'saved_model/bilstm_stacked_i{TIME_STEP}_o{output}' 
# model.save(model_name)